<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=0d17f0db108f9b335f97235a28b37fb150d6d0cbb60cfc92c4025e286df008f1
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-28 11:47:01
-------------------
qualified stocks: 89
with latest results: 28
still star stocks: 17


-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  2.00 K
Current:  1.45 C
-------------------
Today PnL: -42.66 K (-0.29%)
Current PnL: -19.45 L (-12.74%)
CY Booked + Current PnL: -7.35 L (-4.82%)
-------------------
Total profit:  2.39 L
Total loss:  -21.84 L
-------------------
Total Booked + Current PnL: 19.63 L (15.63%)
Total Booked PnL: 39.08 L (31.11%)
Curr Year Booked PnL: 12.10 L (8.36%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 87.34 L (60.35%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.44%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELIANCE,1533.0,-7.66,72.0,X-LC,3.82,231161.0,20795.0,7998.0,-0.15,9.89,3.46,13.68,37.0,2.60,1.60,28.04,XY25,NTT,REFINERIES
79,TTKPRESTIG,770.0,99.57,57.0,M-SC,1.97,85949.0,-14828.0,14921.0,1.41,-14.71,17.36,0.09,245.0,-0.99,0.59,12.92,OX40N,NTT,DURABLES
80,UJJIVANSFB,60.0,131.04,74.0,H-SC,17.52,146232.0,3753.0,15764.0,0.30,2.63,10.78,13.70,163.0,0.24,1.01,72.59,OX40N,NTT,BANKS
2,ABBOTINDIA,35195.0,-9.04,45.0,X-MC,2.98,89790.0,-888.0,15794.0,0.54,-0.98,17.59,16.44,101.0,-0.06,0.62,19.00,X40,ATH,PHARMA
42,ITC,452.0,-37.08,61.0,X-LC,1.56,203864.0,3726.0,17165.0,-0.89,1.86,8.42,10.44,4.0,0.22,1.41,7.53,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.0,-44.35,53.0,X-SC,38.32,54980.0,-10026.0,160954.0,3.21,-15.42,292.75,232.18,198.0,-0.06,0.38,7.24,XY24,NTT,MISC
56,RAJOOENG,143.1,-42.20,46.0,H-SC,18.38,87600.0,-14900.0,55503.0,2.68,-14.54,63.36,39.61,114.0,-0.27,0.61,6.99,AR,ATH,MISC
84,VALIANTORG,838.0,-290.27,44.0,H-SC,8.70,105672.0,-57933.0,179251.0,2.00,-35.41,169.63,74.15,139.0,-0.32,0.73,35.45,XR,NTT,CHEMICALS
62,SAIL,228.0,44.35,54.0,M-MC,9.77,228096.0,3134.0,164526.0,1.97,1.39,72.13,74.53,192.0,0.02,1.58,34.10,XY24,BTT,STEEL
39,INDUSINDBK,1800.0,264.15,64.0,L-MC,9.30,44705.0,-32502.0,57893.0,1.85,-42.10,129.50,32.89,258.0,-0.56,0.31,23.11,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,2096.00,-37.72,36.0,X-SC,4.47,93035.0,-35635.0,80931.0,-3.88,-27.69,86.99,35.20,219.0,-0.44,0.64,6.78,X40,NTT,FOOTWEAR
15,BSOFT,836.99,-14.79,58.0,H-SC,13.98,95956.0,-44697.0,117479.0,-2.69,-31.78,122.43,51.74,131.0,-0.38,0.66,11.02,XR,ATH,IT
32,HINDZINC,730.22,25.96,45.0,M-LC,11.18,205364.0,288.0,112272.0,-2.11,0.14,54.67,54.89,52.0,0.00,1.42,24.27,X5K,ATH,METALS
8,ATULAUTO,844.00,3564.86,46.0,M-SC,3.43,164526.0,-32606.0,122440.0,-1.83,-16.54,74.42,45.57,236.0,-0.27,1.14,16.64,XY24,NTT,AUTO
5,ANGELONE,3033.00,17.35,67.0,X-SC,5.33,207526.0,16520.0,41173.0,-1.79,8.65,19.84,30.21,157.0,0.40,1.43,31.34,X40N,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.00,-30.16,62.0,H-SC,16.43,199601.0,-353.0,97585.0,-0.03,-0.18,48.89,48.63,115.0,-0.00,1.38,11.36,XR,NTT,MISC
57,RECLTD,446.00,45.36,50.0,M-LC,5.49,203775.0,605.0,41529.0,-0.86,0.30,20.38,20.74,55.0,0.01,1.41,7.18,XY25,NTT,FINANCE
88,WIPRO,420.00,-14.70,46.0,M-LC,5.78,150288.0,-657.0,110116.0,-0.62,-0.44,73.27,72.51,53.0,-0.01,1.04,5.56,XR,NTT,IT
32,HINDZINC,730.22,25.96,45.0,M-LC,11.18,205364.0,288.0,112272.0,-2.11,0.14,54.67,54.89,52.0,0.00,1.42,24.27,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,UJJIVANSFB,60.0,131.04,74.0,H-SC,17.52,146232.0,3753.0,15764.0,0.30,2.63,10.78,13.70,163.0,0.24,1.01,72.59,OX40N,NTT,BANKS
79,TTKPRESTIG,770.0,99.57,57.0,M-SC,1.97,85949.0,-14828.0,14921.0,1.41,-14.71,17.36,0.09,245.0,-0.99,0.59,12.92,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.42,50.0,H-SC,2.60,224415.0,-45252.0,81575.0,-1.51,-16.78,36.35,13.47,138.0,-0.55,1.55,14.11,XY24,NTT,PAINTS
74,TATAELXSI,9161.0,-22.20,54.0,H-MC,7.04,105232.0,-26603.0,68832.0,-0.84,-20.18,65.41,32.03,98.0,-0.39,0.73,17.87,OX40N,NTT,IT
18,CERA,9475.0,-24.34,41.0,H-SC,2.62,138115.0,-37788.0,79817.0,0.13,-21.48,57.79,23.89,149.0,-0.47,0.95,19.28,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,UJJIVANSFB,60.00,131.04,74.0,H-SC,17.52,146232.0,3753.0,15764.0,0.30,2.63,10.78,13.70,163.0,0.24,1.01,72.59,OX40N,NTT,BANKS
57,RECLTD,446.00,45.36,50.0,M-LC,5.49,203775.0,605.0,41529.0,-0.86,0.30,20.38,20.74,55.0,0.01,1.41,7.18,XY25,NTT,FINANCE
62,SAIL,228.00,44.35,54.0,M-MC,9.77,228096.0,3134.0,164526.0,1.97,1.39,72.13,74.53,192.0,0.02,1.58,34.10,XY24,BTT,STEEL
32,HINDZINC,730.22,25.96,45.0,M-LC,11.18,205364.0,288.0,112272.0,-2.11,0.14,54.67,54.89,52.0,0.00,1.42,24.27,X5K,ATH,METALS
47,KPIGREEN,731.05,9.02,67.0,H-SC,5.27,129742.0,4445.0,54479.0,-1.41,3.55,41.99,47.03,141.0,0.08,0.90,62.45,MH,ATH,POWER


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.00,-16.77,74.0,H-MC,15.18,214874.0,35066.0,77355.0,0.00,19.50,36.00,62.52,88.0,0.45,1.48,60.24,XR,NTT,BANKS
80,UJJIVANSFB,60.00,131.04,74.0,H-SC,17.52,146232.0,3753.0,15764.0,0.30,2.63,10.78,13.70,163.0,0.24,1.01,72.59,OX40N,NTT,BANKS
87,WHIRLPOOL,2270.00,-38.52,64.0,M-SC,3.18,127420.0,12622.0,81421.0,-0.45,11.00,63.90,81.92,223.0,0.16,0.88,50.85,XR,NTT,DURABLES
47,KPIGREEN,731.05,9.02,67.0,H-SC,5.27,129742.0,4445.0,54479.0,-1.41,3.55,41.99,47.03,141.0,0.08,0.90,62.45,MH,ATH,POWER
37,INDIAMART,4810.62,-53.33,56.0,H-SC,6.89,130258.0,6922.0,124709.0,0.11,5.61,95.74,106.72,119.0,0.06,0.90,30.30,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TATAMOTORS,1065.00,-53.17,19.0,X-LC,16.99,148848.0,-127675.0,237740.0,1.67,-46.17,159.72,39.81,54.0,-0.54,1.03,4.91,XY24,NTT,AUTO
24,DMART,5391.45,-17.31,35.0,X-LC,10.00,101158.0,-1428.0,28233.0,-1.01,-1.39,27.91,26.13,38.0,-0.05,0.70,23.82,X40N,ATH,FMCG
45,JSWINFRA,342.00,-20.69,36.0,X-MC,5.38,199125.0,-1222.0,31721.0,-0.34,-0.61,15.93,15.23,178.0,-0.04,1.38,31.44,X40N,NTT,REALTY
13,BATAINDIA,2096.00,-37.72,36.0,X-SC,4.47,93035.0,-35635.0,80931.0,-3.88,-27.69,86.99,35.20,219.0,-0.44,0.64,6.78,X40,NTT,FOOTWEAR
53,PGHH,17907.65,-30.68,37.0,X-MC,2.89,200730.0,-90.0,67887.0,-0.19,-0.04,33.82,33.76,80.0,-0.00,1.39,5.15,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIEMENS,4671.50,-2.61,47.0,H-LC,1.02,156625.0,-29470.0,76950.0,-0.87,-15.84,49.13,25.51,15.0,-0.38,1.08,15.42,AR,ATH,ELECTRICAL
42,ITC,452.00,-37.08,61.0,X-LC,1.56,203864.0,3726.0,17165.0,-0.89,1.86,8.42,10.44,4.0,0.22,1.41,7.53,X40,NTT,FMCG
53,PGHH,17907.65,-30.68,37.0,X-MC,2.89,200730.0,-90.0,67887.0,-0.19,-0.04,33.82,33.76,80.0,-0.00,1.39,5.15,X40,ATH,FMCG
29,HAVELLS,2069.16,-6.75,42.0,X-MC,3.08,311010.0,-16981.0,123502.0,-0.78,-5.18,39.71,32.48,92.0,-0.14,2.15,5.66,X40,ATH,ELECTRICAL
86,VOLTAS,1530.00,0.88,52.0,X-MC,3.53,211755.0,20013.0,17745.0,-1.73,10.44,8.38,19.69,99.0,1.13,1.46,17.94,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PAGEIND,51769.94,-28.03,37.0,X-MC,6.89,81280.0,-2230.0,22263.0,-0.84,-2.67,27.39,23.99,82.0,-0.10,0.56,2.99,X40,ATH,APPARELS
21,COLPAL,3767.14,-4.76,44.0,X-MC,8.07,222170.0,-41195.0,154541.0,0.23,-15.64,69.56,43.04,84.0,-0.27,1.54,3.14,XY25,ATH,FMCG
10,BAJAJHFL,181.50,-14.90,44.0,X-MC,5.85,179465.0,-21041.0,118196.0,-0.36,-10.49,65.86,48.45,90.0,-0.18,1.24,3.49,X40N,ATH,FINANCE
3,ACC,3906.00,-37.64,51.0,X-MC,2.70,185390.0,-52561.0,205208.0,-0.47,-22.09,110.69,64.15,174.0,-0.26,1.28,3.99,XY24,BTT,CEMENT
85,VBL,671.64,-19.59,47.0,X-LC,9.52,289843.0,-25999.0,138661.0,-1.10,-8.23,47.84,35.67,5.0,-0.19,2.00,4.48,X40N,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-44.35,53.0,X-SC,38.32,54980.0,-10026.0,160954.0,3.21,-15.42,292.75,232.18,198.0,-0.06,0.38,7.24,XY24,NTT,MISC
52,PAGEIND,51769.94,-28.03,37.0,X-MC,6.89,81280.0,-2230.0,22263.0,-0.84,-2.67,27.39,23.99,82.0,-0.10,0.56,2.99,X40,ATH,APPARELS
58,RELAXO,1176.00,-39.97,42.0,X-SC,4.67,81836.0,-63324.0,139252.0,0.08,-43.62,170.16,52.31,136.0,-0.45,0.57,9.54,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-9.04,45.0,X-MC,2.98,89790.0,-888.0,15794.0,0.54,-0.98,17.59,16.44,101.0,-0.06,0.62,19.00,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-37.72,36.0,X-SC,4.47,93035.0,-35635.0,80931.0,-3.88,-27.69,86.99,35.20,219.0,-0.44,0.64,6.78,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4389.97,-27.72,55.0,X-LC,11.67,287527.0,-58429.0,125132.0,-0.85,-16.89,43.52,19.28,1.0,-0.47,1.99,6.29,X40,ATH,IT
40,INFY,2275.00,-18.16,57.0,X-LC,6.57,318648.0,5760.0,165920.0,-0.56,1.84,52.07,54.87,3.0,0.03,2.20,10.28,X40,BTT,IT
42,ITC,452.00,-37.08,61.0,X-LC,1.56,203864.0,3726.0,17165.0,-0.89,1.86,8.42,10.44,4.0,0.22,1.41,7.53,X40,NTT,FMCG
85,VBL,671.64,-19.59,47.0,X-LC,9.52,289843.0,-25999.0,138661.0,-1.10,-8.23,47.84,35.67,5.0,-0.19,2.00,4.48,X40N,ATH,FMCG
1,ABB,7934.00,-40.06,51.0,H-LC,5.97,250344.0,-11275.0,130479.0,-0.43,-4.31,52.12,45.57,7.0,-0.09,1.73,6.52,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7066.67,62.0,L-SC,7.45,78621.0,-15189.0,91806.0,-0.09,-16.19,116.77,81.67,269.0,-0.17,0.54,52.25,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,27.65,81216.0,-32333.0,72388.0,0.21,-28.47,89.13,35.27,268.0,-0.45,0.56,98.78,XR,NTT,HOTELS
50,MASFIN,398.61,-20.51,45.0,H-SC,9.13,90450.0,-7530.0,29134.0,0.03,-7.69,32.21,22.05,152.0,-0.26,0.63,31.48,XR,ATH,FINANCE
87,WHIRLPOOL,2270.00,-38.52,64.0,M-SC,3.18,127420.0,12622.0,81421.0,-0.45,11.00,63.90,81.92,223.0,0.16,0.88,50.85,XR,NTT,DURABLES
47,KPIGREEN,731.05,9.02,67.0,H-SC,5.27,129742.0,4445.0,54479.0,-1.41,3.55,41.99,47.03,141.0,0.08,0.90,62.45,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAMMAANCAP,326.00,-198.86,82.0,M-SC,28.59,170640.0,21420.0,122758.0,0.51,14.35,71.94,96.62,208.0,0.17,1.18,84.67,XY25,NTT,FINANCE
69,SONACOMS,806.63,-31.67,72.0,M-SC,8.29,85006.0,-16153.0,56155.0,0.39,-15.97,66.06,39.54,202.0,-0.29,0.59,19.87,AR,ATH,AUTO
5,ANGELONE,3033.00,17.35,67.0,X-SC,5.33,207526.0,16520.0,41173.0,-1.79,8.65,19.84,30.21,157.0,0.40,1.43,31.34,X40N,NTT,FINANCE
12,BANKINDIA,190.00,-16.77,74.0,H-MC,15.18,214874.0,35066.0,77355.0,0.00,19.50,36.00,62.52,88.0,0.45,1.48,60.24,XR,NTT,BANKS
80,UJJIVANSFB,60.00,131.04,74.0,H-SC,17.52,146232.0,3753.0,15764.0,0.30,2.63,10.78,13.70,163.0,0.24,1.01,72.59,OX40N,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.34
1,25,43.43
2,50,74.15


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.87
LC    31.87
MC    24.30
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.86
X40      16.11
XY25     11.97
XR       11.26
X40N     10.45
AR        7.77
OX40N     7.48
X200      1.74
X5K       1.42
SR        1.08
MH        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.28
X-LC    21.30
X-MC    18.57
M-SC    12.61
M-LC     5.01
X-SC     4.55
H-LC     4.55
H-MC     3.84
M-MC     1.58
L-SC     1.43
L-LC     1.01
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.27,-4.44,38.52
IT,12.38,-17.42,80.39
FINANCE,12.06,-8.42,57.56
MISC,6.85,-15.88,80.09
BANKS,6.62,-4.10,58.87
ELECTRICAL,5.72,-9.05,47.71
PAINTS,5.47,-16.81,34.22
INSURANCE,3.83,0.09,37.16
AUTO,2.76,-44.29,104.51


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3256388.0,22
XR,1318118.0,14
AR,1166726.0,9
X40,826743.0,12
X40N,647294.0,9
OX40N,564890.0,10
XY25,450737.0,8
SR,264933.0,2
X5K,112272.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3337880.0,25
M-SC,1222810.0,15
X-MC,1192836.0,14
X-LC,1058186.0,13
X-SC,520845.0,6
H-MC,324266.0,3
M-LC,284201.0,4
H-LC,279307.0,3
L-SC,254048.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1146132.0      6
           AR         836134.0      5
           XR         779026.0      7
M-SC       XY24       698327.0      6
X-MC       XY24       580605.0      4
X-LC       X40        455784.0      5
           XY24       307481.0      2
X-MC       X40        290028.0      6
X-SC       X40N       278960.0      4
H-SC       SR         264933.0      2
           OX40N      257176.0      4
X-LC       X40N       218417.0      3
H-LC       AR         207429.0      2
H-MC       XY24       178079.0      1
X-MC       XY25       172286.0      2
M-MC       XY24       164526.0      1
L-SC       XR         164194.0      2
X-SC       XY24       160954.0      1
X-MC       X40N       149917.0      2
M-SC       OX40N      149028.0      4
           XR         129534.0      2
           AR         123163.0      2
           XY25       122758.0      1
M-LC       X5K        112272.0      1
           XR         110116.0      1
L-SC       OX40N       89854.0      1
X-SC       X40         80931.0      1
H-MC       XR          77355.0      1
X-LC       XY25        76504.0      3
H-LC       X200        71878.0      1
H-MC       OX40N       68832.0      1
L-MC       XR          57893.0      1
H-SC       MH          54479.0      1
M-LC       XY25        41529.0      1
L-LC       XY25        37660.0      1
M-LC       XY24        20284.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 52.0 seconds
